# IR method experiment

## 目前全部的問題集（共 12 題）

In [1]:
import jieba
import jieba.analyse
jieba.set_dictionary('./jieba_dict/dict.txt.big')

questions = open('./corpus/questions.txt').read()

questions = questions.split('\n')

for i in range(len(questions)):
    print(str(i + 1) + ". " + questions[i])


1. 拿到入學許可後，後續簽證的流程為？
2. 學校附近的住宿相關資訊有哪些呢？
3. 學校附近有什麼地方不建議住的嗎？
4. 學校附近的租金價位？
5. 住宿通常什麼時候要開始找？
6. 住宿的約通常一次簽多久？
7. 住宿簽約時會需要付什麼錢呢？
8. 各地住宿的治安如何？
9. 水電瓦斯費的範圍？
10. 網路要怎麼申裝？
11. 飛到西雅圖後入境的流程？
12. 如何用台灣駕照換發西雅圖駕照？


### 第一句為範例，利用 jieba 做 TFIDF 斷詞後，計算每個詞的 weight

* 斷詞後的結果

In [14]:
seg = jieba.cut(questions[0])

print(",".join(seg))

拿到,入學,許可,後,，,後續,簽證,的,流程,為,？


* 去掉 stopwords（後、的、為、，、？），每個詞在句中 weight

In [3]:
tags = jieba.analyse.extract_tags(questions[0], topK=50, withWeight=True)

for tag, weight in tags:
    print(tag + ": " + str(weight))

入學: 1.99246125048
簽證: 1.99246125048
後續: 1.99246125048
許可: 1.99246125048
流程: 1.29418546015
拿到: 1.1937541929


### 把問題集中剩下的問題都計算出一個詞對應到weight的dictionary

In [4]:
tag_dict = {}
top10_dic = {}

for i in range(len(questions)):
    tags = jieba.analyse.extract_tags(questions[i], topK=50, withWeight=True)
    
    tag_dic = {}
    for (x, w) in tags:
        tag_dic[x] = w
    
    top10_dic[i] = tag_dic

## 以 "想請問錄取後要如何辦赴美簽證？" 為例，查詢問題集中相似的問題

In [5]:
query = "想請問錄取後要如何辦赴美簽證？"

### 將測試句斷詞，並算出斷詞後的weight

In [6]:
tags = jieba.analyse.extract_tags(query, topK=50, withWeight=True)
query_list = []
for x, w in tags:
    print('%s %s' % (x, w))
    query_list = query_list + [(x, w)]

錄取 2.39095350058
請問 2.39095350058
簽證 2.39095350058
赴美 1.82431083177
如何 0.955309795364


### 計算 測試句 和 問題集 中 每個句子的平方差

In [10]:
minimum_file_name = ""
minimum_error = float('inf')
term_dic = {}
error_dic = {}
for i in top10_dic:
    error = 0.0
    tags = top10_dic[i]
    temp_dic = {}
    for j in range(len(query_list)):
        k = query_list[j]
        if k[0] in tags:
            error += (k[1] - tags[k[0]])**2
            temp_dic[k[0]] = tags[k[0]]
        else:
            error += k[1]**2
    
    error_dic[i] = error
    
    if error < minimum_error:
        minimum_error = error
        minimum_file_name = i
        term_dic = temp_dic
        
for i in sorted(error_dic, key=error_dic.get):
    print("Square Error: {:.2f} -> {}".format(error_dic[i], questions[i]))


Square Error: 15.83 -> 拿到入學許可後，後續簽證的流程為？
Square Error: 20.50 -> 如何用台灣駕照換發西雅圖駕照？
Square Error: 20.54 -> 各地住宿的治安如何？
Square Error: 21.39 -> 學校附近的住宿相關資訊有哪些呢？
Square Error: 21.39 -> 學校附近有什麼地方不建議住的嗎？
Square Error: 21.39 -> 學校附近的租金價位？
Square Error: 21.39 -> 住宿通常什麼時候要開始找？
Square Error: 21.39 -> 住宿的約通常一次簽多久？
Square Error: 21.39 -> 住宿簽約時會需要付什麼錢呢？
Square Error: 21.39 -> 水電瓦斯費的範圍？
Square Error: 21.39 -> 網路要怎麼申裝？
Square Error: 21.39 -> 飛到西雅圖後入境的流程？


### 選出 error 最小的句子作為最後的結果

In [13]:
print('Input : ' + query + '\n')

print('Output: ' + questions[minimum_file_name])

Input : 想請問錄取後要如何辦赴美簽證？

Output: 拿到入學許可後，後續簽證的流程為？
